# Brazilian E-Commerce Data Exploration
## Olist Dataset - Bronze Layer Analysis

### 1. Setup e Imports


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

### 2. Carregar dados

In [11]:
df_orders = pd.read_csv('../data/raw/olist_orders_dataset.csv')
df_order_items = pd.read_csv('../data/raw/olist_order_items_dataset.csv')
df_customers = pd.read_csv('../data/raw/olist_customers_dataset.csv')
df_products = pd.read_csv('../data/raw/olist_products_dataset.csv')
df_sellers = pd.read_csv('../data/raw/olist_sellers_dataset.csv')
df_reviews = pd.read_csv('../data/raw/olist_order_reviews_dataset.csv')
df_payments = pd.read_csv('../data/raw/olist_order_payments_dataset.csv')
df_geolocation = pd.read_csv('../data/raw/olist_geolocation_dataset.csv')
df_category_translation = pd.read_csv('../data/raw/product_category_name_translation.csv')
print("✅ Dados carregados com sucesso!")

✅ Dados carregados com sucesso!


### 3. Visão geral dos datasets

In [13]:
print("📊 SHAPE DOS DATASETS\n")
print(f"Orders: {df_orders.shape}")
print(f"Order Items: {df_order_items.shape}")
print(f"Customers: {df_customers.shape}")
print(f"Products: {df_products.shape}")
print(f"Sellers: {df_sellers.shape}")
print(f"Reviews: {df_reviews.shape}")
print(f"Payments: {df_payments.shape}")
print(f"Geolocation: {df_geolocation.shape}")
print(f"Translation: {df_category_translation.shape}")


📊 SHAPE DOS DATASETS

Orders: (99441, 8)
Order Items: (112650, 7)
Customers: (99441, 5)
Products: (32951, 9)
Sellers: (3095, 4)
Reviews: (99224, 7)
Payments: (103886, 5)
Geolocation: (1000163, 5)
Translation: (71, 2)


### 4. Colunas Infos

In [26]:
print("\nINFO ORDERS:")
df_orders.info()



INFO ORDERS:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [ ]:
print("\nINFO CUSTOMERS:")
df_customers.info()


INFO CUSTOMERS:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
